In [9]:
from langchain_community.document_loaders import TextLoader

# Load text
loader = TextLoader("./data/genesi.txt")
content = loader.load()

In [10]:
# Define separators for text
separators = [
   ". ",
   "\n\n",
   "\n",
]

In [11]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Create chunks
splitter = RecursiveCharacterTextSplitter(separators=separators,chunk_size=800, chunk_overlap=200)
docs = splitter.split_documents(content)

In [12]:
from langchain_huggingface import HuggingFaceEmbeddings

# Define and load an embedding model, nomic it's optmized for italian
embeddings_model = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")

In [13]:
from langchain_qdrant import QdrantVectorStore
connection_string = "http://localhost:6333"

# Start embedding
qdrant = QdrantVectorStore.from_documents(
    docs,
    embedding=embeddings_model,
    url=connection_string,
    collection_name="chap02",
    force_recreate=True,
    distance="Cosine"
)

In [14]:
retriever = qdrant.as_retriever(search_kwargs={"k": 4})

In [15]:
retriever.invoke("Testo biblico dove Dio proibisce di mangiare qualcosa")

[Document(metadata={'source': './data/genesi.txt', '_id': '32675a5d-6428-4f1f-89b7-432559582d90', '_collection_name': 'chap02'}, page_content='. 2\xa0E ogni creatura vivente della terra e ogni creatura alata dei cieli, ogni cosa che si muove sul suolo e tutti i pesci del mare avranno paura e terrore di voi. Ora sono nella vostra mano. 3\xa0Tutti gli animali che si muovono e hanno vita vi serviranno da cibo. Ve li do come vi ho dato la verde vegetazione. 4\xa0Ma la carne che ha ancora in sé la vita, il suo sangue, non dovete mangiarla. 5\xa0Inoltre, se il vostro sangue viene versato, ne chiederò conto. Ne chiederò conto a ogni creatura vivente; chiederò conto a ogni uomo della vita di suo fratello. 6\xa0Se qualcuno sparge il sangue di un uomo, il suo sangue sarà sparso da un uomo, perché Dio ha fatto l’uomo a sua immagine'),
 Document(metadata={'source': './data/genesi.txt', '_id': '8daece2d-d965-4df0-b601-bf9fd57edc44', '_collection_name': 'chap02'}, page_content='. 18\xa0Produrrà spin

In [16]:
from numpy import dot
from numpy.linalg import norm

v1 = embeddings_model.embed_query("Dio proibisce di mangiare qualcosa")
v2 = embeddings_model.embed_query("ma il frutto dell'albero della conoscenza del bene e del male non devi mangiarlo")

similarity = dot(v1, v2) / (norm(v1) * norm(v2))
print(similarity)

0.851271792605687
